# Cloud Segmentation - CMX Model (독립 실행 버전)

CMX (Cross-Modal Fusion) 모델을 사용한 구름 세그멘테이션

## 노트북 구조
1. Imports & Config
2. Utility Modules (FRM, FFM)
3. Decoder
4. Backbone
5. CMX Model
6. Augmentations & Dataset
7. Losses & Utils
8. Training & Testing
9. Main Functions
10. 실행

---

In [1]:
!nvidia-smi

Wed Dec 10 08:36:26 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:19:00.0 Off |                  N/A |
|  0%   21C    P5              12W / 250W |   1196MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 1. Imports & Config

In [2]:
import os, sys

print("Working dir:", os.getcwd())
print("Files:", os.listdir())
print("Torch path:", sys.modules.get('torch'))

Working dir: /workspace/basic_code_semina_01/hanbat/CloudSeg
Files: ['utils.py', 'segformer_4ch.ipynb', 'main.py', '__pycache__', 'cache', 'output', 'cs-cmx (2).ipynb', 'segformer_4ch_complete.ipynb', 'clouds-segmentation-2025', 'model.py', 'config.py', 'cloudseg.ipynb', 'submission.csv', 'dataset.py', 'generate_submission.py', 'requirements.txt', 'create_submission_baseline.py', 'test_output.log', 'test.py', 'baseline-clouds-segmentation-kaggle-2025 (3).ipynb', 'requirements_p100_install.sh', 'models', 'sample_submission.csv', 'pretrained', 'train.py', 'losses.py', 'test_code_check.py', 'augmentations.py', '123123123.ipynb', 'cs-cmx (3).ipynb', 'cloud_segmentation_complete.py', 'RGBX_Semantic_Segmentation', 'cloud_segmentation_cmx.ipynb', 'merge_files.py']
Torch path: None


In [ ]:
"""
Cloud Segmentation - CMX Model
"""
import os
import math
import time
import random
import logging
from functools import partial

import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.backends.cudnn as cudnn

import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm.models.layers import trunc_normal_, DropPath, to_2tuple
from transformers import SegformerModel

logger = logging.getLogger(__name__)

# ========== CONFIG ==========
workspace_path = '/kaggle/input/clouds-segmentation-2025' 
output_path = '/kaggle/working'


batch_size = 4


epochs = 60

device = 'cuda' if torch.cuda.is_available() else 'cpu'
patch_size = 512 
patch_stride = 512 // 4 
num_workers = 4 
num_classes = 4 
train_data_rate = 0.8 
loss_func = 'ohem+dice' 
accumulation_steps = 4
cmx_backbone = 'mit_b2'
cmx_pretrained = True
use_timm_pretrained = True

lr_head = 3e-4
lr_backbone = 3e-5
weight_decay = 2e-2 

use_scheduler = True
scheduler_type = "cosine" 
resume = False 
seed = 0     
use_copy_paste = True 

print(f"Device: {device}, Batch: {batch_size}, Epochs: {epochs}")
print(f"Augmentation: Copy-Paste={'ON' if use_copy_paste else 'OFF'}")

print(f"Device: {device}, Batch: {batch_size}, Epochs: {epochs}")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Device: cuda, Batch: 4, Epochs: 60
Augmentation: Copy-Paste=ON
Device: cuda, Batch: 4, Epochs: 60


## 2. Utility Modules (FRM, FFM)

In [4]:
# ========== Feature Rectify Module (FRM) ==========
class ChannelWeights(nn.Module):
    def __init__(self, dim, reduction=1):
        super().__init__()
        self.dim = dim
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Linear(self.dim * 4, self.dim * 4 // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(self.dim * 4 // reduction, self.dim * 2), 
            nn.Sigmoid())

    def forward(self, x1, x2):
        B, _, H, W = x1.shape
        x = torch.cat((x1, x2), dim=1)
        avg = self.avg_pool(x).view(B, self.dim * 2)
        max_val = self.max_pool(x).view(B, self.dim * 2)
        y = torch.cat((avg, max_val), dim=1)
        y = self.mlp(y).view(B, self.dim * 2, 1)
        return y.reshape(B, 2, self.dim, 1, 1).permute(1, 0, 2, 3, 4)


class SpatialWeights(nn.Module):
    def __init__(self, dim, reduction=1):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Conv2d(dim * 2, dim // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim // reduction, 2, kernel_size=1), 
            nn.Sigmoid())

    def forward(self, x1, x2):
        B, _, H, W = x1.shape
        x = torch.cat((x1, x2), dim=1)
        return self.mlp(x).reshape(B, 2, 1, H, W).permute(1, 0, 2, 3, 4)


class FeatureRectifyModule(nn.Module):
    def __init__(self, dim, reduction=1, lambda_c=.5, lambda_s=.5):
        super().__init__()
        self.lambda_c, self.lambda_s = lambda_c, lambda_s
        self.channel_weights = ChannelWeights(dim=dim, reduction=reduction)
        self.spatial_weights = SpatialWeights(dim=dim, reduction=reduction)
    
    def forward(self, x1, x2):
        cw = self.channel_weights(x1, x2)
        sw = self.spatial_weights(x1, x2)
        out_x1 = x1 + self.lambda_c * cw[1] * x2 + self.lambda_s * sw[1] * x2
        out_x2 = x2 + self.lambda_c * cw[0] * x1 + self.lambda_s * sw[0] * x1
        return out_x1, out_x2

FRM = FeatureRectifyModule


# ========== Feature Fusion Module (FFM) ==========
class CrossAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None):
        super().__init__()
        assert dim % num_heads == 0
        self.dim, self.num_heads = dim, num_heads
        self.scale = qk_scale or (dim // num_heads) ** -0.5
        self.kv1 = nn.Linear(dim, dim * 2, bias=qkv_bias)
        self.kv2 = nn.Linear(dim, dim * 2, bias=qkv_bias)

    def forward(self, x1, x2):
        B, N, C = x1.shape
        q1 = x1.reshape(B, -1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        q2 = x2.reshape(B, -1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k1, v1 = self.kv1(x1).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        k2, v2 = self.kv2(x2).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        ctx1 = (k1.transpose(-2, -1) @ v1) * self.scale
        ctx2 = (k2.transpose(-2, -1) @ v2) * self.scale
        x1 = (q1 @ ctx2.softmax(dim=-2)).permute(0, 2, 1, 3).reshape(B, N, C)
        x2 = (q2 @ ctx1.softmax(dim=-2)).permute(0, 2, 1, 3).reshape(B, N, C)
        return x1, x2


class CrossPath(nn.Module):
    def __init__(self, dim, reduction=1, num_heads=None, norm_layer=nn.LayerNorm):
        super().__init__()
        self.channel_proj1 = nn.Linear(dim, dim // reduction * 2)
        self.channel_proj2 = nn.Linear(dim, dim // reduction * 2)
        self.act1, self.act2 = nn.ReLU(inplace=True), nn.ReLU(inplace=True)
        self.cross_attn = CrossAttention(dim // reduction, num_heads=num_heads)
        self.end_proj1 = nn.Linear(dim // reduction * 2, dim)
        self.end_proj2 = nn.Linear(dim // reduction * 2, dim)
        self.norm1, self.norm2 = norm_layer(dim), norm_layer(dim)

    def forward(self, x1, x2):
        y1, u1 = self.act1(self.channel_proj1(x1)).chunk(2, dim=-1)
        y2, u2 = self.act2(self.channel_proj2(x2)).chunk(2, dim=-1)
        v1, v2 = self.cross_attn(u1, u2)
        return self.norm1(x1 + self.end_proj1(torch.cat((y1, v1), dim=-1))), self.norm2(x2 + self.end_proj2(torch.cat((y2, v2), dim=-1)))


class ChannelEmbed(nn.Module):
    def __init__(self, in_channels, out_channels, reduction=1, norm_layer=nn.BatchNorm2d):
        super().__init__()
        self.residual = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.channel_embed = nn.Sequential(
            nn.Conv2d(in_channels, out_channels//reduction, 1, bias=True),
            nn.Conv2d(out_channels//reduction, out_channels//reduction, 3, 1, 1, bias=True, groups=out_channels//reduction),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels//reduction, out_channels, 1, bias=True),
            norm_layer(out_channels))
        self.norm = norm_layer(out_channels)
        
    def forward(self, x, H, W):
        B, N, _C = x.shape
        x = x.permute(0, 2, 1).reshape(B, _C, H, W)
        return self.norm(self.residual(x) + self.channel_embed(x))


class FeatureFusionModule(nn.Module):
    def __init__(self, dim, reduction=1, num_heads=None, norm_layer=nn.BatchNorm2d):
        super().__init__()
        self.cross = CrossPath(dim=dim, reduction=reduction, num_heads=num_heads)
        self.channel_emb = ChannelEmbed(dim*2, dim, reduction, norm_layer)

    def forward(self, x1, x2):
        B, C, H, W = x1.shape
        x1, x2 = x1.flatten(2).transpose(1, 2), x2.flatten(2).transpose(1, 2)
        x1, x2 = self.cross(x1, x2)
        return self.channel_emb(torch.cat((x1, x2), dim=-1), H, W)

FFM = FeatureFusionModule
print("FRM & FFM defined!")

FRM & FFM defined!


## 3. Decoder (MLPDecoder)

In [5]:
class MLP(nn.Module):
    def __init__(self, input_dim=2048, embed_dim=768):
        super().__init__()
        self.proj = nn.Linear(input_dim, embed_dim)

    def forward(self, x):
        return self.proj(x.flatten(2).transpose(1, 2))


class DecoderHead(nn.Module):
    def __init__(self, in_channels=[64, 128, 320, 512], num_classes=40, dropout_ratio=0.1, 
                 norm_layer=nn.BatchNorm2d, embed_dim=768, align_corners=False):
        super().__init__()
        self.num_classes = num_classes
        self.align_corners = align_corners
        self.dropout = nn.Dropout2d(dropout_ratio) if dropout_ratio > 0 else None
        
        c1, c2, c3, c4 = in_channels
        self.linear_c4 = MLP(c4, embed_dim)
        self.linear_c3 = MLP(c3, embed_dim)
        self.linear_c2 = MLP(c2, embed_dim)
        self.linear_c1 = MLP(c1, embed_dim)
        self.linear_fuse = nn.Sequential(
            nn.Conv2d(embed_dim*4, embed_dim, 1), norm_layer(embed_dim), nn.ReLU(inplace=True))
        self.linear_pred = nn.Conv2d(embed_dim, num_classes, 1)
       
    def forward(self, inputs):
        c1, c2, c3, c4 = inputs
        n, _, h, w = c4.shape
        
        _c4 = self.linear_c4(c4).permute(0,2,1).reshape(n, -1, c4.shape[2], c4.shape[3])
        _c4 = F.interpolate(_c4, size=c1.size()[2:], mode='bilinear', align_corners=self.align_corners)
        _c3 = F.interpolate(self.linear_c3(c3).permute(0,2,1).reshape(n, -1, c3.shape[2], c3.shape[3]), size=c1.size()[2:], mode='bilinear', align_corners=self.align_corners)
        _c2 = F.interpolate(self.linear_c2(c2).permute(0,2,1).reshape(n, -1, c2.shape[2], c2.shape[3]), size=c1.size()[2:], mode='bilinear', align_corners=self.align_corners)
        _c1 = self.linear_c1(c1).permute(0,2,1).reshape(n, -1, c1.shape[2], c1.shape[3])
        
        _c = self.linear_fuse(torch.cat([_c4, _c3, _c2, _c1], dim=1))
        return self.linear_pred(self.dropout(_c) if self.dropout else _c)

print("Decoder defined!")

Decoder defined!


## 4. Backbone (MiT Transformer)

In [6]:
class DWConv(nn.Module):
    def __init__(self, dim=768):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        return self.dwconv(x.permute(0, 2, 1).reshape(B, C, H, W)).flatten(2).transpose(1, 2)


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x, H, W):
        x = self.fc1(x)
        x = self.dwconv(x, H, W)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        return self.drop(x)


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1):
        super().__init__()
        self.dim, self.num_heads = dim, num_heads
        self.scale = qk_scale or (dim // num_heads) ** -0.5
        self.q = nn.Linear(dim, dim, bias=qkv_bias)
        self.kv = nn.Linear(dim, dim * 2, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.sr_ratio = sr_ratio
        if sr_ratio > 1:
            self.sr = nn.Conv2d(dim, dim, kernel_size=sr_ratio, stride=sr_ratio)
            self.norm = nn.LayerNorm(dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        if self.sr_ratio > 1:
            x_ = self.norm(self.sr(x.permute(0, 2, 1).reshape(B, C, H, W)).reshape(B, C, -1).permute(0, 2, 1))
            kv = self.kv(x_).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        else:
            kv = self.kv(x).reshape(B, -1, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        k, v = kv[0], kv[1]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = self.attn_drop(attn.softmax(dim=-1))
        x = self.proj_drop(self.proj((attn @ v).transpose(1, 2).reshape(B, N, C)))
        return x


class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., 
                 attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        self.mlp = Mlp(in_features=dim, hidden_features=int(dim * mlp_ratio), act_layer=act_layer, drop=drop)

    def forward(self, x, H, W):
        x = x + self.drop_path(self.attn(self.norm1(x), H, W))
        return x + self.drop_path(self.mlp(self.norm2(x), H, W))


class OverlapPatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.proj(x)
        _, _, H, W = x.shape
        return self.norm(x.flatten(2).transpose(1, 2)), H, W

print("Backbone components defined!")

Backbone components defined!


In [7]:
class RGBXTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dims=[64, 128, 256, 512], 
                 num_heads=[1, 2, 4, 8], mlp_ratios=[4, 4, 4, 4], qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, norm_fuse=nn.BatchNorm2d,
                 depths=[3, 4, 6, 3], sr_ratios=[8, 4, 2, 1]):
        super().__init__()
        self.depths = depths
        
        # RGB patch embeds
        self.patch_embed1 = OverlapPatchEmbed(img_size, 7, 4, in_chans, embed_dims[0])
        self.patch_embed2 = OverlapPatchEmbed(img_size // 4, 3, 2, embed_dims[0], embed_dims[1])
        self.patch_embed3 = OverlapPatchEmbed(img_size // 8, 3, 2, embed_dims[1], embed_dims[2])
        self.patch_embed4 = OverlapPatchEmbed(img_size // 16, 3, 2, embed_dims[2], embed_dims[3])
        
        # NIR patch embeds
        self.extra_patch_embed1 = OverlapPatchEmbed(img_size, 7, 4, in_chans, embed_dims[0])
        self.extra_patch_embed2 = OverlapPatchEmbed(img_size // 4, 3, 2, embed_dims[0], embed_dims[1])
        self.extra_patch_embed3 = OverlapPatchEmbed(img_size // 8, 3, 2, embed_dims[1], embed_dims[2])
        self.extra_patch_embed4 = OverlapPatchEmbed(img_size // 16, 3, 2, embed_dims[2], embed_dims[3])
        
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        
        # Stage 1
        self.block1 = nn.ModuleList([Block(embed_dims[0], num_heads[0], mlp_ratios[0], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[0]) for i in range(depths[0])])
        self.norm1 = norm_layer(embed_dims[0])
        self.extra_block1 = nn.ModuleList([Block(embed_dims[0], num_heads[0], mlp_ratios[0], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[0]) for i in range(depths[0])])
        self.extra_norm1 = norm_layer(embed_dims[0])
        cur += depths[0]
        
        # Stage 2
        self.block2 = nn.ModuleList([Block(embed_dims[1], num_heads[1], mlp_ratios[1], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur], norm_layer=norm_layer, sr_ratio=sr_ratios[1]) for i in range(depths[1])])
        self.norm2 = norm_layer(embed_dims[1])
        self.extra_block2 = nn.ModuleList([Block(embed_dims[1], num_heads[1], mlp_ratios[1], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur+1], norm_layer=norm_layer, sr_ratio=sr_ratios[1]) for i in range(depths[1])])
        self.extra_norm2 = norm_layer(embed_dims[1])
        cur += depths[1]
        
        # Stage 3
        self.block3 = nn.ModuleList([Block(embed_dims[2], num_heads[2], mlp_ratios[2], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[2]) for i in range(depths[2])])
        self.norm3 = norm_layer(embed_dims[2])
        self.extra_block3 = nn.ModuleList([Block(embed_dims[2], num_heads[2], mlp_ratios[2], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[2]) for i in range(depths[2])])
        self.extra_norm3 = norm_layer(embed_dims[2])
        cur += depths[2]
        
        # Stage 4
        self.block4 = nn.ModuleList([Block(embed_dims[3], num_heads[3], mlp_ratios[3], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[3]) for i in range(depths[3])])
        self.norm4 = norm_layer(embed_dims[3])
        self.extra_block4 = nn.ModuleList([Block(embed_dims[3], num_heads[3], mlp_ratios[3], qkv_bias, qk_scale,
            drop_rate, attn_drop_rate, dpr[cur + i], norm_layer=norm_layer, sr_ratio=sr_ratios[3]) for i in range(depths[3])])
        self.extra_norm4 = norm_layer(embed_dims[3])
        
        # FRM & FFM
        self.FRMs = nn.ModuleList([FRM(dim=embed_dims[i], reduction=1) for i in range(4)])
        self.FFMs = nn.ModuleList([FFM(dim=embed_dims[i], reduction=1, num_heads=num_heads[i], norm_layer=norm_fuse) for i in range(4)])

    def forward_features(self, x_rgb, x_e):
        B = x_rgb.shape[0]
        outs = []
        
        # Stage 1
        x_rgb, H, W = self.patch_embed1(x_rgb)
        x_e, _, _ = self.extra_patch_embed1(x_e)
        for blk in self.block1: x_rgb = blk(x_rgb, H, W)
        for blk in self.extra_block1: x_e = blk(x_e, H, W)
        x_rgb, x_e = self.norm1(x_rgb), self.extra_norm1(x_e)
        x_rgb = x_rgb.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_e = x_e.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_rgb, x_e = self.FRMs[0](x_rgb, x_e)
        outs.append(self.FFMs[0](x_rgb, x_e))
        
        # Stage 2
        x_rgb, H, W = self.patch_embed2(x_rgb)
        x_e, _, _ = self.extra_patch_embed2(x_e)
        for blk in self.block2: x_rgb = blk(x_rgb, H, W)
        for blk in self.extra_block2: x_e = blk(x_e, H, W)
        x_rgb, x_e = self.norm2(x_rgb), self.extra_norm2(x_e)
        x_rgb = x_rgb.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_e = x_e.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_rgb, x_e = self.FRMs[1](x_rgb, x_e)
        outs.append(self.FFMs[1](x_rgb, x_e))
        
        # Stage 3
        x_rgb, H, W = self.patch_embed3(x_rgb)
        x_e, _, _ = self.extra_patch_embed3(x_e)
        for blk in self.block3: x_rgb = blk(x_rgb, H, W)
        for blk in self.extra_block3: x_e = blk(x_e, H, W)
        x_rgb, x_e = self.norm3(x_rgb), self.extra_norm3(x_e)
        x_rgb = x_rgb.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_e = x_e.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_rgb, x_e = self.FRMs[2](x_rgb, x_e)
        outs.append(self.FFMs[2](x_rgb, x_e))
        
        # Stage 4
        x_rgb, H, W = self.patch_embed4(x_rgb)
        x_e, _, _ = self.extra_patch_embed4(x_e)
        for blk in self.block4: x_rgb = blk(x_rgb, H, W)
        for blk in self.extra_block4: x_e = blk(x_e, H, W)
        x_rgb, x_e = self.norm4(x_rgb), self.extra_norm4(x_e)
        x_rgb = x_rgb.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_e = x_e.reshape(B, H, W, -1).permute(0, 3, 1, 2)
        x_rgb, x_e = self.FRMs[3](x_rgb, x_e)
        outs.append(self.FFMs[3](x_rgb, x_e))
        
        return outs

    def forward(self, x_rgb, x_e):
        return self.forward_features(x_rgb, x_e)

print("RGBXTransformer defined!")

RGBXTransformer defined!


In [8]:
# MiT Backbone Variants
class mit_b1(RGBXTransformer):
    def __init__(self, fuse_cfg=None, **kwargs):
        super().__init__(patch_size=4, embed_dims=[64, 128, 320, 512], num_heads=[1, 2, 5, 8], mlp_ratios=[4, 4, 4, 4],
            qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[2, 2, 2, 2], sr_ratios=[8, 4, 2, 1],
            drop_rate=0.0, drop_path_rate=0.1)

class mit_b2(RGBXTransformer):
    def __init__(self, fuse_cfg=None, **kwargs):
        super().__init__(patch_size=4, embed_dims=[64, 128, 320, 512], num_heads=[1, 2, 5, 8], mlp_ratios=[4, 4, 4, 4],
            qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[3, 4, 6, 3], sr_ratios=[8, 4, 2, 1],
            drop_rate=0.0, drop_path_rate=0.1)

class mit_b3(RGBXTransformer):
    def __init__(self, fuse_cfg=None, **kwargs):
        super().__init__(patch_size=4, embed_dims=[64, 128, 320, 512], num_heads=[1, 2, 5, 8], mlp_ratios=[4, 4, 4, 4],
            qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[3, 4, 18, 3], sr_ratios=[8, 4, 2, 1],
            drop_rate=0.0, drop_path_rate=0.1)

class mit_b4(RGBXTransformer):
    def __init__(self, fuse_cfg=None, **kwargs):
        super().__init__(patch_size=4, embed_dims=[64, 128, 320, 512], num_heads=[1, 2, 5, 8], mlp_ratios=[4, 4, 4, 4],
            qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), depths=[3, 8, 27, 3], sr_ratios=[8, 4, 2, 1],
            drop_rate=0.0, drop_path_rate=0.1)


def load_pretrained_from_transformers(model, backbone='mit_b2'):
    """Hugging Face transformers에서 SegFormer pretrained 가중치 로드"""
    hf_names = {'mit_b1': 'nvidia/mit-b1', 'mit_b2': 'nvidia/mit-b2', 'mit_b3': 'nvidia/mit-b3', 'mit_b4': 'nvidia/mit-b4'}
    hf_name = hf_names.get(backbone, 'nvidia/mit-b2')
    print(f"Loading pretrained weights from: {hf_name}...")
    
    try:
        hf_model = SegformerModel.from_pretrained(hf_name)
        raw_state_dict = hf_model.state_dict()
        state_dict = {}
        kv_weights, kv_biases = {}, {}
        
        for k, v in raw_state_dict.items():
            new_key = k.replace('encoder.', '')
            for i in range(4):
                new_key = new_key.replace(f'patch_embeddings.{i}', f'patch_embed{i+1}')
                new_key = new_key.replace(f'block.{i}.', f'block{i+1}.')
                new_key = new_key.replace(f'layer_norm.{i}', f'norm{i+1}')
            new_key = new_key.replace('attention.self.query', 'attn.q')
            new_key = new_key.replace('attention.output.dense', 'attn.proj')
            new_key = new_key.replace('attention.self.sr', 'attn.sr')
            new_key = new_key.replace('attention.self.layer_norm', 'attn.norm')
            new_key = new_key.replace('output.dense', 'mlp.fc2')
            new_key = new_key.replace('intermediate.dense', 'mlp.fc1')
            new_key = new_key.replace('dwconv.dwconv', 'mlp.dwconv.dwconv')
            
            if 'attention.self.key' in k:
                base = new_key.replace('attention.self.key', 'attn.kv')
                kv_weights.setdefault(base, {})['key'] = v
                continue
            elif 'attention.self.value' in k:
                base = new_key.replace('attention.self.value', 'attn.kv')
                kv_weights.setdefault(base, {})['value'] = v
                continue
            
            if any(x in new_key for x in ['patch_embed', 'block', 'norm']):
                state_dict[new_key] = v
                if 'patch_embed' in new_key:
                    state_dict[new_key.replace('patch_embed', 'extra_patch_embed')] = v
                elif 'block' in new_key:
                    state_dict[new_key.replace('block', 'extra_block')] = v
                elif 'norm' in new_key and 'attn' not in new_key:
                    state_dict[new_key.replace('norm', 'extra_norm')] = v
        
        for base, kv in kv_weights.items():
            if 'key' in kv and 'value' in kv:
                w = torch.cat([kv['key'], kv['value']], dim=0)
                state_dict[base] = w
                state_dict[base.replace('block', 'extra_block')] = w
        
        model_state = model.state_dict()
        loaded = 0
        for k, v in state_dict.items():
            if k in model_state and model_state[k].shape == v.shape:
                model_state[k] = v
                loaded += 1
        model.load_state_dict(model_state, strict=False)
        print(f"✓ Loaded {loaded} keys from pretrained!")
        del hf_model
    except Exception as e:
        print(f"Warning: Could not load pretrained: {e}")

print("MiT variants & pretrained loader defined!")

MiT variants & pretrained loader defined!


## 5. CMX Model

In [9]:
class CMXModel(nn.Module):
    """CMX: Cross-Modal Fusion for RGB-X Semantic Segmentation"""
    def __init__(self, backbone='mit_b2', num_classes=4, decoder_embed_dim=512, 
                 pretrained_model=None, norm_layer=nn.BatchNorm2d):
        super().__init__()
        backbones = {'mit_b1': mit_b1, 'mit_b2': mit_b2, 'mit_b3': mit_b3, 'mit_b4': mit_b4}
        self.backbone = backbones.get(backbone, mit_b2)(norm_fuse=norm_layer)
        channels = [64, 128, 320, 512]  # mit_b1, mit_b2, mit_b3, mit_b4 모두 동일한 채널 수
        self.decode_head = DecoderHead(in_channels=channels, num_classes=num_classes, 
                                        norm_layer=norm_layer, embed_dim=decoder_embed_dim)
    
    def forward(self, rgb, nir):
        if nir.shape[1] == 1:
            nir = nir.repeat(1, 3, 1, 1)
        features = self.backbone(rgb, nir)
        return self.decode_head(features)


def create_model(device, num_classes=4, cmx_backbone='mit_b2', cmx_pretrained=None, use_timm_pretrained=True):
    model = CMXModel(backbone=cmx_backbone, num_classes=num_classes)
    if use_timm_pretrained and cmx_pretrained is None:
        load_pretrained_from_transformers(model.backbone, cmx_backbone)
    model.to(device)
    return model


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("CMX Model defined!")

CMX Model defined!


## 6. Augmentations & Dataset

In [10]:


# =========================================================
#  1. Copy-Paste Augmentation (완벽함)
# =========================================================
class CopyPasteAugmentation:
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, rgb, nir, mask, src_rgb, src_nir, src_mask):
        if random.random() > self.p:
            return rgb, nir, mask

        H, W = rgb.shape[:2]

        # binary cloud
        cloud = (src_mask > 0).astype(np.uint8)
        if cloud.sum() == 0:
            return rgb, nir, mask

        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(cloud, connectivity=8)
        if num_labels <= 1:
            return rgb, nir, mask

        # pick random component
        comp_idx = random.randint(1, num_labels - 1)
        x, y, w, h, area = stats[comp_idx]
        if area < 100:
            return rgb, nir, mask

        comp = (labels == comp_idx).astype(np.uint8)
        crop_rgb = src_rgb[y:y+h, x:x+w].copy()
        crop_nir = src_nir[y:y+h, x:x+w].copy()
        crop_mask = src_mask[y:y+h, x:x+w].copy()
        crop_comp = comp[y:y+h, x:x+w].copy()

        # resize
        scale = random.uniform(0.4, 1.2)
        nh, nw = int(h * scale), int(w * scale)
        if nh < 2 or nw < 2:
            return rgb, nir, mask
        
        # 크기가 원본 이미지를 초과하지 않도록 제한
        nh = min(nh, H)
        nw = min(nw, W)

        crop_rgb = cv2.resize(crop_rgb, (nw, nh))
        crop_nir = cv2.resize(crop_nir, (nw, nh))
        crop_mask = cv2.resize(crop_mask, (nw, nh), interpolation=cv2.INTER_NEAREST)
        crop_comp = cv2.resize(crop_comp, (nw, nh), interpolation=cv2.INTER_NEAREST)

        # ensure shape (H,W,1)
        if crop_nir.ndim == 2:
            crop_nir = crop_nir[..., None]

        # 안전한 위치 계산
        px = random.randint(0, max(0, W - nw))
        py = random.randint(0, max(0, H - nh))
        
        # 실제 붙여넣을 영역 크기 계산 (경계 체크)
        paste_h = min(nh, H - py)
        paste_w = min(nw, W - px)
        
        # crop도 같은 크기로 자르기
        crop_rgb = crop_rgb[:paste_h, :paste_w]
        crop_nir = crop_nir[:paste_h, :paste_w]
        crop_mask = crop_mask[:paste_h, :paste_w]
        crop_comp = crop_comp[:paste_h, :paste_w]
        
        region = crop_comp.astype(bool)

        # apply
        rgb_patch = rgb[py:py+paste_h, px:px+paste_w]
        nir_patch = nir[py:py+paste_h, px:px+paste_w]
        mask_patch = mask[py:py+paste_h, px:px+paste_w]

        rgb_patch[region] = crop_rgb[region]
        nir_patch[region] = crop_nir[region]
        mask_patch[region] = crop_mask[region]

        return rgb, nir, mask
# =========================================================
#  2. Train Transforms (완벽함)
# =========================================================
class ImageAug:
    def __init__(self, crop_size=512):
        # 1. 기하학적 변환 (4채널 지원) - RGB+NIR 동시 적용
        self.geom = A.Compose([
            A.RandomCrop(height=crop_size, width=crop_size, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(
                shift_limit=0.05, 
                scale_limit=0.2, 
                rotate_limit=30,
                border_mode=0,
                p=0.5
            ),
            A.OneOf([
                A.GridDistortion(p=0.5),
                A.OpticalDistortion(distort_limit=0.5, p=0.5),
            ], p=0.3),
        ])
        
        # 2. 색상 변환 (3채널만 지원) - RGB에만 적용
        self.color = A.Compose([
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
            A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=20, val_shift_limit=15, p=0.5),
            A.CLAHE(clip_limit=2.0, p=0.3),
        ])

        # 3. 정규화 (RGB용)
        self.normalize_rgb = A.Compose([
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ])
        
        # 4. 정규화 (NIR용)
        self.normalize_nir = A.Compose([
            A.Normalize(
                mean=[0.5],
                std=[0.25],
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ])

    def __call__(self, rgb, nir, mask):
        if nir.ndim == 2:
            nir = nir[..., None]

        # Step 1: 기하학적 변환 (RGB+NIR+Mask 동시 적용)
        combined = np.concatenate([rgb, nir], axis=-1)
        aug = self.geom(image=combined, mask=mask)
        combined = aug["image"]
        mask = aug["mask"]
        
        # 분리
        rgb_aug = combined[:, :, :3]
        nir_aug = combined[:, :, 3:]

        # Step 2: RGB에만 색상 변환 적용
        rgb_aug = self.color(image=rgb_aug)["image"]

        # Step 3: 정규화 및 텐서 변환
        rgb_t = self.normalize_rgb(image=rgb_aug)["image"]
        nir_t = self.normalize_nir(image=nir_aug)["image"]
        mask_t = torch.from_numpy(mask).long()

        return rgb_t, nir_t, mask_t

class DefaultAug:
    def __init__(self):
        self.tf = A.Compose([
            A.Normalize(
                mean=[0.485, 0.456, 0.406, 0.5],
                std=[0.229, 0.224, 0.225, 0.25],
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ])

    def __call__(self, rgb, nir, mask):
        if nir.ndim == 2:
            nir = nir[..., None]

        combined = np.concatenate([rgb, nir], axis=-1)

        aug = self.tf(image=combined, mask=mask)
        img4 = aug["image"]
        mask = aug["mask"]

        rgb_t = img4[:3, :, :]
        nir_t = img4[3:4, :, :]
        mask_t = mask.long()

        return rgb_t, nir_t, mask_t
# =========================================================
#  4. Dataset
# =========================================================
class CloudDataset(torch.utils.data.Dataset):
    def __init__(self, rgb_paths, ngr_paths, label_paths, 
                 is_train=True, crop_size=512, use_copy_paste=False):

        self.is_train = is_train
        self.use_copy_paste = use_copy_paste
        self.copy_paste = CopyPasteAugmentation()
        
        # [수정] 나중에 파일명을 알기 위해 경로 저장
        self.rgb_paths_list = rgb_paths

        # 원본 이미지 로드 (RAM에 올림)
        self.rgb_imgs = [cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB) for p in rgb_paths]
        self.nir_imgs = [cv2.imread(p)[:, :, 2] for p in ngr_paths]
        
        # 라벨이 있는 경우에만 로드 (Test셋은 라벨이 없음)
        if len(label_paths) > 0:
            self.lbl_imgs = [cv2.imread(p) for p in label_paths]
            self.has_label = True
        else:
            self.lbl_imgs = []
            self.has_label = False

        self.train_tf = ImageAug(crop_size)
        self.val_tf = DefaultAug()

    def __len__(self):
        return len(self.rgb_imgs)

    def __getitem__(self, idx):
        # 1. 이미지 가져오기
        rgb = self.rgb_imgs[idx].copy()
        nir = self.nir_imgs[idx].copy()[..., None]  # (H, W) -> (H, W, 1)
        if not self.has_label:
            dummy = np.zeros(rgb.shape[:2], dtype=np.uint8)
            rgb_t, nir_t, _ = self.val_tf(rgb, nir, dummy)
            return (rgb_t, nir_t), self.rgb_paths_list[idx]

        # -----------------------------------------------
        # Case A: 학습/검증 (라벨 있음)
        # -----------------------------------------------
        if self.has_label:
            lbl = self.lbl_imgs[idx].copy()
            
            # 마스크 생성
            mask = np.zeros(lbl.shape[:2], dtype=np.uint8)
            mask[np.all(lbl == [0,0,255], axis=-1)] = 1  # Thick
            mask[np.all(lbl == [0,255,0], axis=-1)] = 2  # Thin
            mask[np.all(lbl == [0,255,255], axis=-1)] = 3 # Shadow

            # Copy-Paste
            if self.is_train and self.use_copy_paste:
                j = random.randint(0, len(self.rgb_imgs)-1)
                src_rgb = self.rgb_imgs[j]
                src_nir = self.nir_imgs[j][..., None]
                # 소스 이미지의 라벨도 가져와야 함
                src_lbl = self.lbl_imgs[j]
                src_mask = np.zeros(src_lbl.shape[:2], dtype=np.uint8)
                src_mask[np.all(src_lbl == [0,0,255], axis=-1)] = 1
                src_mask[np.all(src_lbl == [0,255,0], axis=-1)] = 2
                src_mask[np.all(src_lbl == [0,255,255], axis=-1)] = 3
                
                rgb, nir, mask = self.copy_paste(rgb, nir, mask, src_rgb, src_nir, src_mask)

            # Transform
            if self.is_train:
                rgb_t, nir_t, mask_t = self.train_tf(rgb, nir, mask)
            else:
                rgb_t, nir_t, mask_t = self.val_tf(rgb, nir, mask)
            
            return (rgb_t, nir_t), mask_t

        # -----------------------------------------------
        # Case B: 테스트 (라벨 없음)
        # -----------------------------------------------
        else:
            # 테스트용 Transform (기하학적 변환 없이 정규화만)
            # 마스크가 없으므로 더미 마스크(0)를 만들어서 넘김
            dummy_mask = np.zeros(rgb.shape[:2], dtype=np.uint8)
            rgb_t, nir_t, _ = self.val_tf(rgb, nir, dummy_mask)
            
            # [중요] 마스크 대신 '파일 경로'를 반환
            return (rgb_t, nir_t), self.rgb_paths_list[idx]

print("Dataset updated for Test Support!")

Dataset updated for Test Support!


## 7. Losses & Utils

In [11]:
# =========================================================
# 1. Loss Functions (Type & Dimension Safe)
# =========================================================

def ohem_ce_loss(preds, targets, ratio=0.25, ignore_index=255):
    targets = targets.long()
    if targets.dim() == 4:
        targets = targets.squeeze(1)

    n, c, h, w = preds.shape
    preds = preds.permute(0,2,3,1).reshape(-1, c)
    targets = targets.view(-1)

    ce = F.cross_entropy(preds, targets, ignore_index=ignore_index, reduction='none')
    ce_sorted, idx = torch.sort(ce, descending=True)

    num_hard = int(ce.numel() * ratio)
    ce_hard = ce_sorted[:num_hard]

    return ce_hard.mean()
    
def dice_loss(preds, targets, eps=1e-7):
    """
    Dice Loss
    preds: (B, C, H, W) - Logits
    targets: (B, H, W) or (B, 1, H, W) - Indices
    """
    # 1. Targets 안전 처리
    targets = targets.long()
    if targets.dim() == 4:
        targets = targets.squeeze(1) # (B, 1, H, W) -> (B, H, W)
    
    num_classes = preds.shape[1]
    
    # 2. One-hot Encoding
    # targets: (B, H, W) -> one_hot: (B, H, W, C) -> permute: (B, C, H, W)
    true_1_hot = F.one_hot(targets, num_classes).permute(0, 3, 1, 2).float()
    
    # 3. Softmax
    probas = F.softmax(preds, dim=1)
    
    # 4. Calc Dice
    dims = (0,) + tuple(range(2, targets.ndimension() + 1))
    
    intersection = torch.sum(probas * true_1_hot, dims)
    cardinality = torch.sum(probas + true_1_hot, dims)
    
    return 1 - (2. * intersection / (cardinality + eps)).mean()


def jaccard_loss(preds, targets, eps=1e-7):
    """Jaccard (IoU) Loss"""
    targets = targets.long()
    if targets.dim() == 4:
        targets = targets.squeeze(1)
        
    num_classes = preds.shape[1]
    true_1_hot = F.one_hot(targets, num_classes).permute(0, 3, 1, 2).float()
    probas = F.softmax(preds, dim=1)
    
    dims = (0,) + tuple(range(2, targets.ndimension() + 1))
    
    intersection = torch.sum(probas * true_1_hot, dims)
    cardinality = torch.sum(probas + true_1_hot, dims)
    
    return 1 - (intersection / (cardinality - intersection + eps)).mean()


def ce_loss(preds, targets, ignore=255):
    """Cross-Entropy Loss"""
    targets = targets.long()
    if targets.dim() == 4:
        targets = targets.squeeze(1)
        
    return F.cross_entropy(preds, targets, ignore_index=ignore)


def get_loss_function(name):
    """Loss 함수 이름으로 함수 반환 (인자 순서 통일됨)"""
    if name == 'dice':
        return dice_loss
    elif name == 'jaccard':
        return jaccard_loss
    elif name == 'ce':
        return ce_loss
    elif name in ['dice+ce']:
        def loss(preds, targets):
            return 0.7 * ce_loss(preds, targets) + 0.3 * dice_loss(preds, targets)
        return loss
    elif name in ['dice+jaccard', 'dice + jaccard']:
        def combined_loss(preds, targets):
            return dice_loss(preds, targets) + jaccard_loss(preds, targets)
        return combined_loss
    elif name in ['ohem+dice']:
        def loss(preds, targets):
            return 0.7 * ohem_ce_loss(preds, targets, ratio=0.25) + \
                   0.3 * dice_loss(preds, targets)
        return loss
    else:
        print(f"Warning: Unknown loss name '{name}'. Using Dice Loss.")
        return dice_loss

# =========================================================
# 2. Utils & Metrics
# =========================================================

def init_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False


def fitness_test(true, pred, num_classes=4):
    """mIOU, Pixel Accuracy, Dice Score 계산"""
    eps = 1e-7
    
    # [수정] 차원 및 타입 안전 처리
    true = true.long()
    if true.dim() == 4:
        true = true.squeeze(1)
        
    # Pred: (B, C, H, W) -> (B, H, W) (argmax)
    if pred.shape[1] > 1: # Logits이 들어오면 argmax
        pred_max = pred.argmax(dim=1)
    else:
        pred_max = pred.squeeze(1) # 이미 argmax된 상태라면
    
    # Pixel Accuracy
    pix_acc = (true == pred_max).float().mean()
    
    # One-hot 변환 for IoU/Dice calculation
    true_1h = F.one_hot(true, num_classes).permute(0, 3, 1, 2).float()
    pred_1h = F.one_hot(pred_max, num_classes).permute(0, 3, 1, 2).float()
    
    dims = (0,) + tuple(range(2, true.ndimension() + 1))
    
    # Intersection & Union
    inter = torch.sum(pred_1h * true_1h, dims) # 교집합
    union = torch.sum(pred_1h + true_1h, dims) - inter # 합집합
    
    # mIoU
    m_iou = (inter / (union + eps)).mean()
    
    # Dice Score
    dice = (2. * inter / (torch.sum(pred_1h + true_1h, dims) + eps)).mean()
    
    return m_iou.item(), pix_acc.item(), dice.item()


def mask2rle(img):
    '''
    베이스라인 로직:
    img: 3채널 컬러 이미지 (BGR)
    img.T.flatten(): 이미지를 전치(Transpose) 후 1차원으로 폄
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

print("Losses & Utils defined (Fixed Version)!")

Losses & Utils defined (Fixed Version)!


## 8. Training

In [12]:
import os
import math
import time
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
def visualize_predictions(model, val_dl, device, save_dir, epoch, num_samples=5):
    """
    검증 데이터셋의 일부를 추론하여 RGB, NIR, Pred, GT를 시각화하고 저장합니다.
    기본적으로 5세트를 출력합니다.
    """
    model.eval()
    os.makedirs(save_dir, exist_ok=True)

    # 정규화 역변환 상수
    MEAN = np.array([0.485, 0.456, 0.406])
    STD = np.array([0.229, 0.224, 0.225])
    NIR_MEAN = 0.5
    NIR_STD = 0.25

    count = 0
    with torch.no_grad():
        for imgs, targets in val_dl:
            rgb, nir = imgs
            rgb, nir = rgb.to(device), nir.to(device)

            if targets.dim() == 4:
                targets = targets.squeeze(1)
            gt_batch = targets.long().cpu().numpy()

            preds = model(rgb, nir)
            # 해상도 복원
            if preds.shape[-2:] != targets.shape[-2:]:
                preds = F.interpolate(preds, size=targets.shape[-2:], mode='bilinear', align_corners=False)
            
            pred_batch = preds.argmax(dim=1).cpu().numpy()

            # 배치 내 이미지 순회
            for i in range(rgb.shape[0]):
                if count >= num_samples:
                    return # 목표 샘플 수(5개) 채우면 종료

                # -----------------------------
                # 1. 이미지 복원
                # -----------------------------
                img_rgb = rgb[i].permute(1, 2, 0).cpu().numpy()
                img_rgb = img_rgb * STD + MEAN
                img_rgb = np.clip(img_rgb, 0, 1)

                img_nir = nir[i].squeeze().cpu().numpy()
                img_nir = img_nir * NIR_STD + NIR_MEAN
                img_nir = np.clip(img_nir, 0, 1)

                img_gt = gt_batch[i]
                img_pred = pred_batch[i]

                # -----------------------------
                # 2. Plotting (5장 모두 출력)
                # -----------------------------
                fig, ax = plt.subplots(1, 4, figsize=(16, 4))
                
                ax[0].imshow(img_rgb)
                ax[0].set_title(f"RGB")
                ax[0].axis("off")

                ax[1].imshow(img_nir, cmap='gray')
                ax[1].set_title(f"NIR")
                ax[1].axis("off")

                ax[2].imshow(img_pred, cmap='jet', vmin=0, vmax=3)
                ax[2].set_title(f"Prediction")
                ax[2].axis("off")

                ax[3].imshow(img_gt, cmap='jet', vmin=0, vmax=3)
                ax[3].set_title(f"GT Mask")
                ax[3].axis("off")

                plt.suptitle(f"Epoch {epoch} - Sample {count+1}", fontsize=14)
                plt.tight_layout()

                save_name = os.path.join(save_dir, f"epoch_{epoch}_sample_{count}.png")
                plt.savefig(save_name)
                
                plt.show()
                
                count += 1

def train_one_epoch(model, optimizer, data_loader, loss_fn, device, epoch, num_epochs, accumulation_steps=1):
    model.train()
    losses = []
    
    optimizer.zero_grad()
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), desc=f'Epoch {epoch}/{num_epochs}')
    
    for i, (imgs, targets) in pbar:
        rgb, nir = imgs 
        rgb, nir, targets = rgb.to(device), nir.to(device), targets.to(device)
        
        if targets.dim() == 4:
            targets = targets.squeeze(1)
        targets = targets.long()
        
        preds = model(rgb, nir)

        # 해상도 맞추기
        if preds.shape[2:] != targets.shape[1:]:
            preds = F.interpolate(preds, size=targets.shape[1:], mode='bilinear', align_corners=False)
        
        loss = loss_fn(preds, targets)
        
        # --- Gradient Accumulation ---
        loss = loss / accumulation_steps
        loss.backward()
        
        # 🔥 여기서 실제로 파라미터 업데이트
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            optimizer.step()
            optimizer.zero_grad()
        # -----------------------------

        current_loss = loss.item() * accumulation_steps
        losses.append(current_loss)
        pbar.set_postfix({'loss': f'{current_loss:.4f}'})
    
    return np.mean(losses)

def val_one_epoch(model, data_loader, device, epoch, num_epochs):
    model.eval()
    metrics = {'miou': [], 'acc': [], 'dice': []}
    pbar = tqdm(data_loader, desc=f'Val {epoch}/{num_epochs}')
    
    for imgs, targets in pbar:
        rgb, nir = imgs
        rgb, nir, targets = rgb.to(device), nir.to(device), targets.to(device)
        
        if targets.dim() == 4: targets = targets.squeeze(1)
        targets = targets.long()
        
        with torch.no_grad():
            preds = model(rgb, nir)
            if preds.shape[2:] != targets.shape[1:]:
                preds = F.interpolate(preds, size=targets.shape[1:], mode='bilinear', align_corners=False)
            
            try:
                m_iou, pix_acc, dice = fitness_test(targets.unsqueeze(1), preds)
                metrics['miou'].append(m_iou)
                metrics['acc'].append(pix_acc)
                metrics['dice'].append(dice)
                pbar.set_postfix({'mIOU': f'{m_iou:.4f}', 'Acc': f'{pix_acc:.4f}'})
            except NameError:
                pbar.set_postfix({'Status': 'Metric func missing'})

    if len(metrics['miou']) == 0: return {'mIOU': 0, 'Accuracy': 0, 'Dice': 0}

    return {
        'mIOU': np.mean(metrics['miou']), 
        'Accuracy': np.mean(metrics['acc']), 
        'Dice': np.mean(metrics['dice'])
    }

import gc



def train(model, optimizer, train_dl, val_dl, loss_func, epochs, device, 
          use_scheduler=False, save_path='./ckpt', scheduler_type='cosine', 
          val_interval=10, accumulation_steps=1): # [수정] 인자 추가
    
    torch.cuda.empty_cache()
    loss_fn = get_loss_function(loss_func)
    
    os.makedirs(save_path, exist_ok=True)
    visual_save_dir = os.path.join(save_path, "visuals")
    
    scheduler = None
    if use_scheduler:
        if scheduler_type == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
        elif scheduler_type == "plateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    best_fit = 0.0
    weight_file = os.path.join(save_path, 'cmx_best.pt')

    print(f"Start Training... Total Epochs: {epochs} (Val every {val_interval})")
    print(f"Gradient Accumulation: {accumulation_steps} steps (Effective Batch: {train_dl.batch_size * accumulation_steps})")

    for epoch in range(epochs):
        # 1. Train
        # [수정] accumulation_steps를 train_one_epoch에 전달
        train_loss = train_one_epoch(model, optimizer, train_dl, loss_fn, device, epoch, epochs, accumulation_steps)
        
        # Scheduler
        if scheduler:
            if scheduler_type == "plateau":
                scheduler.step(train_loss)
            else:
                scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        print(f"\n[Epoch {epoch}] Loss: {train_loss:.4f} | LR: {current_lr:.2e}")

        # 2. Validation (Interval)
        if (epoch + 1) % val_interval == 0 or (epoch + 1) == epochs:
            print(f"    >>> Running Validation at Epoch {epoch}...")
            val_metrics = val_one_epoch(model, val_dl, device, epoch, epochs)
            
            print(f"    Val mIOU: {val_metrics['mIOU']:.4f} | Acc: {val_metrics['Accuracy']:.4f}")

            # 5개 샘플 시각화 호출
            visualize_predictions(model, val_dl, device, visual_save_dir, epoch, num_samples=5)

            # Best Model Save
            current_score = val_metrics['mIOU']
            if current_score > best_fit:
                best_fit = current_score
                torch.save({
                    'epoch': epoch,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'best_score': val_metrics, # 딕셔너리 저장 (Load 에러 방지)
                }, weight_file)
                print(f"    >>> Best Model Saved! (mIOU: {best_fit:.4f})")
        
        print("-" * 50)

    print(f"Training Completed. Best mIOU: {best_fit:.4f}")
    return best_fit
print("All functions updated successfully!")

All functions updated successfully!


## 9. Testing & Submission

In [13]:
def load_best_model(model, device, output_path):
    """최고 성능 모델 로드 (CMX 전용)"""
    save_path = os.path.join(output_path, 'ckpt')
    checkpoint_path = os.path.join(save_path, 'cmx_best.pt')
    
    if not os.path.exists(checkpoint_path):
        print(f"Warning: Model not found at {checkpoint_path}")
        return model
        
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    print('Model load success')
    return model

In [14]:


def predict_test_set(model, test_dataloader, device, result_path):
    model.eval()
    os.makedirs(result_path, exist_ok=True)
    
    print(f"Generating predictions to {result_path}...")
    
    with torch.no_grad():
        # CMX Dataset은 ((rgb, nir), path)를 반환함
        for batch_idx, (imgs, img_paths) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
            rgb, nir = imgs
            rgb, nir = rgb.to(device), nir.to(device)
            
            # 1. 모델 추론
            preds = model(rgb, nir)
            
            # 2. 원본 크기 복원 (베이스라인은 원본 크기로 저장함)
            # 이미지 경로에서 원본 이미지 크기를 읽어와서 맞춤
            original_img = cv2.imread(img_paths[0])
            h_orig, w_orig = original_img.shape[:2]
            
            if preds.shape[2:] != (h_orig, w_orig):
                preds = F.interpolate(preds, size=(h_orig, w_orig), mode='bilinear', align_corners=False)
            
            # 3. Argmax로 클래스 인덱스 추출 (Batch, H, W)
            _, idx_mask = preds.max(1) 
            
            # 4. 배치 내 이미지 처리 (Batch Size가 1이라도 루프 유지)
            for i in range(rgb.shape[0]):
                # 빈 컬러 이미지 생성 (H, W, 3)
                pred_img = np.zeros((h_orig, w_orig, 3), dtype=np.uint8)
                
                # GPU 텐서를 numpy로 변환
                mask = idx_mask[i].cpu().numpy()
                
                # [핵심] 베이스라인과 동일한 BGR 색상 매핑
                # 0: Background (Black) - [0, 0, 0] (이미 0으로 초기화됨)
                pred_img[mask == 1] = [0, 0, 255]    # Thick Cloud (Red)
                pred_img[mask == 2] = [0, 255, 0]    # Thin Cloud (Green)
                pred_img[mask == 3] = [0, 255, 255]  # Cloud Shadow (Yellow)
                
                # 5. 이미지 저장
                filename = os.path.basename(img_paths[i])
                cv2.imwrite(os.path.join(result_path, filename), pred_img)
def create_submission(result_path, workspace_path):
    print("Creating submission.csv...")
    
    # 결과 폴더의 파일 리스트 (정렬 필수)
    test_label_file_list = sorted(os.listdir(result_path))
    test_label_path_list = [os.path.join(result_path, x) for x in test_label_file_list]
    
    rle_list = []
    # 이미지를 하나씩 읽어서 RLE 인코딩
    for file_path in tqdm(test_label_path_list, desc="RLE Encoding"):
        img = cv2.imread(file_path) # BGR로 읽기
        rle = mask2rle(img)         # 위에서 정의한 mask2rle 함수 사용
        rle_list.append(rle)
    
    # DataFrame 생성
    submission = pd.DataFrame({
        'Image_Label': test_label_file_list,
        'EncodedPixels': rle_list
    })
    
    submission.to_csv('submission.csv', index=False)
    print("submission.csv saved successfully!")
    print(submission.head())
    return submission
    
print("Testing functions defined!")

Testing functions defined!


## 10. Main Functions

In [15]:
def load_data():
    """데이터 경로 로드"""

    rgb_path = os.path.join(workspace_path, 'train/rgb/')
    ngr_path = os.path.join(workspace_path, 'train/ngr/')
    label_path = os.path.join(workspace_path, 'train/label/')
    
    rgb_images = sorted([os.path.join(rgb_path, x) for x in os.listdir(rgb_path)])
    ngr_images = sorted([os.path.join(ngr_path, x) for x in os.listdir(ngr_path)])
    label_images = sorted([os.path.join(label_path, x) for x in os.listdir(label_path)])
    
    return rgb_images, ngr_images, label_images


def create_dataloaders(rgb_images, ngr_images, label_images):
    """데이터로더 생성 (New CloudDataset 호환)"""
    train_split = int(len(rgb_images) * train_data_rate)
    
    # 1. Train Dataset
    # patch_size -> crop_size로 이름 변경됨
    # transforms 객체를 밖에서 넣지 않고 내부에서 생성함
    train_ds = CloudDataset(
        rgb_paths=rgb_images[:train_split], 
        ngr_paths=ngr_images[:train_split], 
        label_paths=label_images[:train_split],
        is_train=True, 
        crop_size=patch_size, 
        use_copy_paste=use_copy_paste # 전역 변수 or 인자 활용
    )
    
    # 2. Val Dataset
    val_ds = CloudDataset(
        rgb_paths=rgb_images[train_split:], 
        ngr_paths=ngr_images[train_split:], 
        label_paths=label_images[train_split:],
        is_train=False
        # crop_size, use_copy_paste 필요 없음
    )
    
    # 3. DataLoaders
    # Validation은 batch_size=1을 권장 (검증 시 원본 크기 유지 등 이슈 최소화)
    train_dl = torch.utils.data.DataLoader(
        train_ds, batch_size=batch_size, shuffle=True, 
        num_workers=num_workers, pin_memory=True, drop_last=True
    )
    val_dl = torch.utils.data.DataLoader(
        val_ds, batch_size=1, shuffle=False, 
        num_workers=num_workers, pin_memory=True, drop_last=True
    )
    
    return train_dl, val_dl


def create_test_dataloader():
    """테스트 데이터로더 (New CloudDataset 호환)"""
    # 테스트 경로 설정
    test_rgb_dir = os.path.join(workspace_path, 'test/rgb')
    test_ngr_dir = os.path.join(workspace_path, 'test/ngr')
    
    test_rgb = sorted([os.path.join(test_rgb_dir, x) for x in os.listdir(test_rgb_dir)])
    test_ngr = sorted([os.path.join(test_ngr_dir, x) for x in os.listdir(test_ngr_dir)])
    
    # 주의: CloudDataset은 label_paths가 필수이므로, 더미 라벨(또는 빈 리스트) 처리 필요
    # 현재 CloudDataset 구조상 빈 리스트 []를 넣으면 __getitem__에서 에러날 수 있음.
    # 테스트용이라 라벨이 없다면, 아래처럼 Dataset 클래스를 살짝 수정하거나 
    # 더미 경로를 넣어줘야 하는데, 일단은 빈 리스트로 호출해봅니다.
    # (만약 에러나면 Test 전용 Dataset 클래스를 분리해야 함)
    
    test_ds = CloudDataset(
        rgb_paths=test_rgb, 
        ngr_paths=test_ngr, 
        label_paths=[], # 라벨 없음
        is_train=False
    )
    
    return torch.utils.data.DataLoader(
        test_ds, batch_size=1, shuffle=False, 
        num_workers=num_workers, pin_memory=True
    )

print("Main functions updated for new CloudDataset!")

Main functions updated for new CloudDataset!


## 11. 실행

### 11.1 데이터 로드 및 모델 생성

In [16]:
# 시드 초기화
init_seeds(seed)

# 데이터 로드
print("Loading data...")
rgb_images, ngr_images, label_images = load_data()
print(f"  RGB: {len(rgb_images)}, NGR: {len(ngr_images)}, Label: {len(label_images)}")

# 데이터로더 생성
print("\nCreating dataloaders...")
train_dl, val_dl = create_dataloaders(rgb_images, ngr_images, label_images)
print(f"Train batches: {len(train_dl)}, Val batches: {len(val_dl)}")

# 모델 생성
print("\nCreating model...")
model = create_model(device, num_classes, cmx_backbone, cmx_pretrained, use_timm_pretrained)
print(f"  Backbone: {cmx_backbone}")
print(f"  Parameters: {count_parameters(model):,}")


optimizer = torch.optim.AdamW([
    {"params": model.backbone.parameters(), "lr": lr_backbone},
    {"params": model.decode_head.parameters(), "lr": lr_head},
], weight_decay=weight_decay)

print(f"  Optimizer: AdamW initialized with separated LR")
print(f"   - Backbone LR: {lr_backbone:.1e}")
print(f"   - Head LR:     {lr_head:.1e}")

Loading data...
  RGB: 829, NGR: 829, Label: 829

Creating dataloaders...


/opt/conda/lib/python3.11/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Train batches: 165, Val batches: 166

Creating model...
  Backbone: mit_b2
  Parameters: 66,562,956
  Optimizer: AdamW initialized with separated LR
   - Backbone LR: 3.0e-05
   - Head LR:     3.0e-04


### 11.2 학습 실행

In [17]:
# 학습 시작
print("\nStarting training...")
best_metrics = train(
    model=model, 
    optimizer=optimizer, 
    train_dl=train_dl, 
    val_dl=val_dl, 
    loss_func=loss_func, 
    epochs=epochs, 
    device=device,
    use_scheduler=use_scheduler,
    save_path=os.path.join(output_path, 'ckpt'),
    val_interval=5,
    accumulation_steps=4
)


Starting training...
Start Training... Total Epochs: 60 (Val every 5)
Gradient Accumulation: 4 steps (Effective Batch: 16)


Epoch 0/60:   5%|▌         | 9/165 [00:26<07:42,  2.97s/it, loss=1.9491]


KeyboardInterrupt: 

In [ ]:
print(f"\nBest Results:")
if isinstance(best_metrics, dict):
    print(f"  mIOU: {best_metrics.get('mIOU', 0):.4f}")
    print(f"  Accuracy: {best_metrics.get('Accuracy', 0):.4f}")
    print(f"  Dice: {best_metrics.get('Dice', 0):.4f}")
else:

    print(f"  mIOU (Main Score): {best_metrics:.4f}")

### 11.3 테스트 및 제출

In [ ]:
# 베스트 모델 로드
model = load_best_model(model, device, output_path)

# 테스트 데이터로더
test_dl = create_test_dataloader()

# 예측
result_path = os.path.join(output_path, 'results')
predict_test_set(model, test_dl, device, result_path)

# 제출 파일 생성
submission = create_submission(result_path, workspace_path)
print(submission.head())
print("\nDone!")